In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Data

In [ ]:
data = pd.read_excel('Data/input-10_buses.xlsx', None)
prices = pd.read_excel('Data/prices_average_month.xlsx', None)
for month, prices_data in prices.items():
    prices[month] = prices_data.loc[prices_data.index.repeat(4)]
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
scenarios = range(1,5)

# Optimisation Routine

In [ ]:
def optimisation_model(scenario, month):
    model = pyo.ConcreteModel()
    i = len(data['Trip time']['Time begin (min)'])
    t = len(data['Prices']['Time'])
    k = len(data['Buses']['Bus (kWh)'])
    n = len(data['Chargers']['Charger (kWh/min)'])

    T_start = data['Trip time']['Time begin (min)'].tolist()
    T_start = [int(x) for x in T_start]
    T_end = data['Trip time']['Time finish (min)'].tolist()
    T_end = [int(x) for x in T_end]

    alpha = data['Chargers']['Charger (kWh/min)'].tolist()
    beta = data['Chargers']['Charger (kWh/min)'].tolist()
    ch_eff = 0.90
    dch_eff = 1/0.9
    gama = data['Energy consumption']['Energy consumption (kWh/km*min)'].tolist()

    P = prices[month]['Price'].tolist()
    S = np.array(P) * 0.75

    E_0 = 0.2
    E_min = 0.2
    E_max = 1
    E_end = 0.2
    C_bat = data['Buses']['Bus (kWh)'].tolist()

    if scenario == 4:
        R = 73
    else:
        R = 130
    
    Ah = data['Buses']['Ah (Ah)'].tolist()
    V = 512
    T = 96
    delta_t = 4

    model.I = pyo.RangeSet(i) # set of trips
    model.T = pyo.RangeSet(t) # set of timesteps
    model.K = pyo.RangeSet(k) # set of buses
    model.N = pyo.RangeSet(n) # set of chargers

    model.T_start = pyo.Param(model.I, initialize=lambda model, i: T_start[i-1]) # start time of trip i
    model.T_end = pyo.Param(model.I, initialize=lambda model, i: T_end[i-1]) # end time of trip i
    model.alpha = pyo.Param(model.N, initialize=lambda model, n: alpha[n-1]) # charging power of charger n
    model.beta = pyo.Param(model.N, initialize=lambda model, n: beta[n-1]) # discharging power of charger n
    model.ch_eff = pyo.Param(initialize=ch_eff) # charging efficiency of charger n
    model.dch_eff = pyo.Param(initialize=dch_eff) # discharging efficiency of charger n
    model.gama = pyo.Param(model.I, initialize=lambda model, i: gama[i-1]) # energy consumption
    model.P = pyo.Param(model.T, initialize=lambda model, t: P[t-1]) # electricity purchasing price in time t
    model.S = pyo.Param(model.T, initialize=lambda model, t: S[t-1]) # electricity selling price in time t
    model.E_0 = pyo.Param(initialize=E_0) # initial energy level of bus k
    model.E_min = pyo.Param(initialize=E_min) # minimum energy level allowed for bus k
    model.E_max = pyo.Param(initialize=E_max) # maximum energy level allowed for bus k
    model.E_end = pyo.Param(initialize=E_end) # minimum energy after an operation day for bus k
    model.C_bat = pyo.Param(model.K, initialize=lambda model, k: C_bat[k-1]) # total capacity of the bus k battery
    model.R = pyo.Param(initialize=R) # battery replacement costs of the bus k
    model.Ah = pyo.Param(model.K, initialize=lambda model, k: Ah[k-1]) # energy consumed until EOL of bus k
    model.V = pyo.Param(initialize=V) # operational voltage of charger n

    #binary variables
    # binary variable indicating if bus k is serving trip i at time t
    model.b = pyo.Var(model.K,model.I, model.T, within=pyo.Binary)
    # binary variable indicating if bus k is occupying a charger n at time t to charge
    model.x = pyo.Var(model.K, model.N, model.T, domain=pyo.Binary)
    
    if scenario != 1:
    # binary variable indicating if bus k is occupying a charger n at time t to discharge
        model.y = pyo.Var(model.K, model.N, model.T, domain=pyo.Binary)

    #non-negative variables
    # energy level of bus k at time t
    model.e = pyo.Var(model.K, model.T, within=pyo.NonNegativeReals) 
    # electricity purchased from the grid at time t
    model.w_buy = pyo.Var(model.T, within=pyo.NonNegativeReals)
    # electricity sold to the grid at time t
    model.w_sell = pyo.Var(model.T, within=pyo.NonNegativeReals)
    
    if scenario != 1:
        # total degradation cost of the bus k battery at time t
        model.d = pyo.Var(model.K, model.T, within=pyo.NonNegativeReals)

    model.constraints = pyo.ConstraintList()  # Create a set of constraints

    if scenario == 1:
        #constraint 6
        for k in model.K:
            for t in model.T:
                model.constraints.add(sum(model.b[k,i,t] for i in model.I) + sum(model.x[k,n,t] for n in model.N) <=1)
    else:
        #constraint 6
        for k in model.K:
            for t in model.T:
                model.constraints.add(sum(model.b[k,i,t] for i in model.I) + sum(model.x[k,n,t] for n in model.N) + sum(model.y[k,n,t] for n in model.N) <=1)
    
    #constraint 7
    for i in model.I: 
        for t in range(model.T_start[i],model.T_end[i]):
            model.constraints.add(sum(model.b[k,i,t] for k in model.K) == 1)
    
    #constraint 8
    for i in model.I:
        for k in model.K:
            for t in range(model.T_start[i],model.T_end[i]-1):
                model.constraints.add(model.b[k,i,t+1] >= model.b[k,i,t])
    
    if scenario == 1:
        #constraint 9
        for n in model.N:
            for t in model.T:
                model.constraints.add(sum(model.x[k,n,t] for k in model.K) <= 1)
    else:
        #constraint 9
        for n in model.N:
            for t in model.T:
                model.constraints.add(sum(model.x[k,n,t] for k in model.K) + sum(model.y[k,n,t] for k in model.K) <= 1)
    
    if scenario == 1:
        #constraint 10
        for k in model.K:
            for t in range(2,T+1):
                model.constraints.add(model.e[k,t] == model.e[k,t-1] + sum(model.ch_eff*model.alpha[n]*model.x[k,n,t] for n in model.N) - sum(model.gama[i]*model.b[k,i,t] for i in model.I))
    else:
        #constraint 10
        for k in model.K:
            for t in range(2,T+1):
                model.constraints.add(model.e[k,t] == model.e[k,t-1] + sum(model.ch_eff*model.alpha[n]*model.x[k,n,t] for n in model.N) - sum(model.gama[i]*model.b[k,i,t] for i in model.I) - sum(model.dch_eff*model.beta[n]*model.y[k,n,t] for n in model.N))
    
    #constraint 11
    for t in model.T:
        model.constraints.add(sum(model.ch_eff*model.alpha[n]*model.x[k,n,t] for n in model.N for k in model.K) == model.w_buy[t])
    
    if scenario != 1:
        #constraint 12
        for t in model.T:
            model.constraints.add(sum(model.dch_eff*model.beta[n]*model.y[k,n,t] for n in model.N for k in model.K) == model.w_sell[t])       
    
    #constraint 13
    for k in model.K:
        for t in model.T:
            model.constraints.add(model.e[k,t] >= model.C_bat[k] * model.E_min)
            model.constraints.add(E_max * model.C_bat[k] >= model.e[k,t] + sum(model.ch_eff*model.alpha[n]*model.x[k,n,t] for n in model.N))          
    
    #constraint 14
    for k in model.K:
        model.constraints.add(model.e[k,1] == model.E_0*model.C_bat[k])
    
    if scenario != 1:
        #constraint 15
        for k in model.K:
            for t in model.T:
                model.constraints.add(model.d[k,t] == ((model.R*model.C_bat[1]*1000)/(delta_t*model.Ah[k]*model.V))* sum(model.beta[n]*model.y[k,n,t] for n in model.N))

    if scenario == 1:
        def rule_obj(mod):
            return sum(mod.P[t]*mod.w_buy[t] for t in mod.T)
        model.obj = pyo.Objective(rule=rule_obj, sense=pyo.minimize)

    if scenario == 2:
        def rule_obj(mod):
            return sum(mod.P[t]*mod.w_buy[t] for t in mod.T) - sum(mod.S[t]*mod.w_sell[t] for t in mod.T)
        model.obj = pyo.Objective(rule=rule_obj, sense=pyo.minimize)

    if scenario == 3:
        def rule_obj(mod):
            return sum(mod.P[t]*mod.w_buy[t] for t in mod.T) - sum(mod.S[t]*mod.w_sell[t] for t in mod.T) + sum(mod.d[k,t] for k in mod.K for t in mod.T)
        model.obj = pyo.Objective(rule=rule_obj, sense=pyo.minimize)

    if scenario == 4:
        def rule_obj(mod):
            return sum(mod.P[t]*mod.w_buy[t] for t in mod.T) - sum(mod.S[t]*mod.w_sell[t] for t in mod.T) + sum(mod.d[k,t] for k in mod.K for t in mod.T)
        model.obj = pyo.Objective(rule=rule_obj, sense=pyo.minimize)

    print('Solving scenario', scenario, 'for month:', month)
    opt = pyo.SolverFactory('gurobi')
    opt.options['timelimit'] = 120
    opt.options['mipgap'] = 0.005
    opt.solve(model,tee=False)
    print('Total costs', model.obj())

    return model

# Solution Loop

In [ ]:
for scenario in scenarios:
    results = {'Month': [], 'Degradation Costs (€)': [], 'Energy Sold (€)': [], 'Energy Bought (€)': []}
    degrad =  {'Month': [], 'Degradation per bus': []}
    for month in months:
        model = optimisation_model(scenario, month)

        if scenario != 1:
            degra_value = pyo.value(sum(model.d[k, t] for k in model.K for t in model.T))
        else:
            degra_value = None

        if scenario != 1:
            sell = pyo.value(sum(model.S[t] * model.w_sell[t] for t in model.T))
        else:
            sell = None

        buy = pyo.value(sum(model.P[t] * model.w_buy[t] for t in model.T))

        # Append results for each month
        results['Month'].append(month)
        results['Degradation Costs (€)'].append(degra_value)
        results['Energy Sold (€)'].append(sell)
        results['Energy Bought (€)'].append(buy)
        
        if scenario != 1:
            # degradation costs for each bus
            d_values = np.array([[sum(model.d[k, t].value for t in model.T)] for k in model.K])
            degrad['Month'].append(month)
            degrad['Degradation per bus'].append(d_values)

    
    if scenario != 1:
        # Save to Excel
        file = f'degradation_per_bus_scenario{scenario}.xlsx'
        with pd.ExcelWriter(file, engine='xlsxwriter') as writer:
            for month, values in zip(degrad['Month'], degrad['Degradation per bus']):
                # Flatten the 2-dimensional array
                flattened_values = np.ravel(values)

                # Create a DataFrame for each month
                df = pd.DataFrame({'Degradation per bus': flattened_values})
                
                # Write the DataFrame to the Excel sheet named as the month
                df.to_excel(writer, sheet_name=month, index=False)

    # Create DataFrame
    df = pd.DataFrame(results)

    # Save to Excel
    file_name = f'scenario_{scenario}.xlsx'
    with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Results', index=False)        